# imports


In [42]:
import pandas as pd
from openpyxl import load_workbook
import shutil
import openpyxl
import ast
import random
from IPython.display import display, HTML
from difflib import get_close_matches

In [ ]:
 pandas, openpyxl, shutil, ast, random, IPython.display, difflib

# inputs

# The steps starts from here
### 1. input an excel file with student names and their free time written in time intervals with title "stufree.xlsx"
- column names are fname, lname, and free
- time intervals : hour to  hour
- for example studentA is free from 2 to 3. It should be written as 2-3


In [43]:
studentsfree = pd.read_excel("stufree.xlsx")
studentsfree.head()

,fname,lname,free
0,Ashlyn,Reich,10-2
1,Aurelio,Molina,10-3
2,Avril,Boling,10-3
3,Belva,Daugherty,4-5
4,Brandee,Howerton,2-5



### 2. input an excel file with student names that companies selected with title "company_list.xlsx"
- column names are fname and lname the sheet names (company names) will have the name of the company


In [44]:
company_list = pd.read_excel("company_list.xlsx")
company_list.head()

,fname,lname,Time
0,Saran,Albers,NaN
1,Sibyl,Peacock,NaN
2,Stephany,Willard,NaN
3,Taryn,Choate,NaN
4,Vanetta,Forbes,NaN



### 3. input an excel file student names  and rest columns are time slots 10 - 4:30. file name is "sample_book.xlsx"
- column names are fname and lname all the time slots 
- change the students first name and last name to real studnets names and leave time slots as they are.

In [45]:
sample = pd.read_excel("sample_book.xlsx")
sample.head()

,fname,lname,10,10:30,11,11:30,12,12:30,1,1:30,2,2:30,3,3:30,4,4:30
0,Ashlyn,Reich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aurelio,Molina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Avril,Boling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Belva,Daugherty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brandee,Howerton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
def get_student_available_time_slot():
    stu_excel_file = pd.read_excel("stufree.xlsx")
    
    # holds all the intervals students have    
    free_time_interval = list(stu_excel_file['free'])
    
    # holds all slots students have  
    get_student_available_time_slot_arr = []
    
    all_slots = ["10", "10:30", "11", "11:30", "12", "12:30", "1", "1:30", "2", "2:30", "3", "3:30", "4", "4:30", "5:00"]

    for i in free_time_interval:
        
        i = i.replace(" ", "")
        i = i.replace(".", "")
        i = i.replace("230", "2:30")
        i = i.replace("330", "3:30")
#       i = i.replace("5", "4:30")
        i = i.replace("5", "5:00")
        #more than interval
        if "and" in i:
            i = i.replace("and", "--")
            
        # one interval           
        x = i.split("-")
        if len(x) > 2: 
           
            st = f'{all_slots[all_slots.index(x[0]):all_slots.index(x[1])]} and  {all_slots[all_slots.index(x[3]) : all_slots.index(x[4])]}  '
 
            get_student_available_time_slot_arr.append(st)
            
            
        else:
            get_student_available_time_slot_arr.append(all_slots[all_slots.index(x[0]):all_slots.index(x[1])])
    
    return get_student_available_time_slot_arr 

get_student_available_time_slot_arr = get_student_available_time_slot()


In [47]:
def fix_free_to_slots():
    shutil.copy("stufree.xlsx", "stufreeEdited.xlsx")
    writer = pd.ExcelWriter('stufreeEdited.xlsx', engine='openpyxl') 
    df = pd.read_excel("stufree.xlsx");
    wb  = writer.book
    df.insert(3, 'Time_array', get_student_available_time_slot_arr,  allow_duplicates = False)
    df.to_excel(writer, index=False)
    wb.save('stufreeEdited.xlsx')
fix_free_to_slots()

In [48]:
stu_excel_file = pd.read_excel("stufree.xlsx")
def get_stu_fullname_array(stu):
    # create an array of the students full name
    fname = stu['fname']
    lname = stu['lname']
    all_student = [] 
    for f, l in zip(fname, lname):
       all_student.append(f + " " + l)
    return all_student
stu_excel_file = pd.read_excel("stufree.xlsx")
all_students = get_stu_fullname_array(stu_excel_file)


def create_dict_fullname_time():
    stu = pd.read_excel("stufreeEdited.xlsx")
    stu_dict ={}
    count = 0
    for i, j , h in zip(stu['fname'], stu['lname'], stu['Time_array']):
        name = i+ " "+ j
        stu_dict[name] = h

    return stu_dict


# since we are done with students time slots

# companies

In [49]:
# scheduilng and removing form the list 
def scheduling(companyList, company_ava_dic, i, stu_dict):
    if i not in company_ava_dic.values():

        if len(companyList[i])> 0:
            indexofchonesn = companyList[i].index(random.choice(companyList[i]))
            if isinstance(stu_dict[i], list):
                company_ava_dic[stu_dict[i].pop(0)]= i
            else:

                company_ava_dic[list(stu_dict[i]).pop(0)]= i
            
     
# get the company list of students and the times that company has for interivews 
# return student dict that has 
# keys --> time 
# values --> students name
def TieringCompany(companyList, company_ava_dic, stu_dict):
    #Tier A
    for i in companyList:
        if len(companyList[i]) <= 1  :
            scheduling(companyList, company_ava_dic, i, stu_dict)

    for i in companyList:
        if len(companyList[i]) >=2  and len(companyList[i]) <= 5 :
            scheduling(companyList, company_ava_dic, i, stu_dict)
    
    for i in companyList:
        if len(companyList[i]) >= 6 and len(companyList[i]) <= 10 :
            scheduling(companyList,company_ava_dic, i, stu_dict)

    for i in companyList:
        if  len(companyList[i]) > 10 :
            scheduling(companyList, company_ava_dic, i, stu_dict)
    return company_ava_dic



# return the max row and student names object

In [50]:

def get_max_row_col_comp_stu(sheet):
    return sheet.max_row, sheet['A2':f'B{sheet.max_row}']


#  Get the student full name from the company work book -- done

In [51]:
# get list of obj student names from the company 
# return list of string full names 
def get_company_student_fullname(company_student):
    company_stu_names = []
    for row in company_student:
        if row[0].value != None and row[1].value != None :
            company_stu_names.append(row[0].value + " " + row[1].value )
    return company_stu_names


#  Get ride the student big array from the company workbook  because they were schedule -- done

In [52]:
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]


def convert_comp_stud_tuble_list_name(company_students):
    company_stu = []
    for i in company_students:
        if i[0].value is not None:
            s = str(i[0].value + " " + i[1].value)
            company_stu.append( s)
    return company_stu
        
def remove_student_from_all_student_arr(all_students, company_students):
    for i , j in zip(company_students, all_students):
        if i in all_students:
            all_students.remove(i)
    return all_students


In [53]:
def remove_empty(li):
    for i in li:
        if len(i) <5 or i == "''":
            li.remove(i)
    
    

    li = list(filter(lambda x: x!= '', li))
    return li
def appendvalues(li, dic, sheet, wb, ws_dict, left):

    li = remove_empty(li)

    counter = 0
    count_stu_toadd = 8 - len(ws_dict['fname'])
    for i in li:

        if counter  < count_stu_toadd: 

            counter = counter + 1
            
            fname = i.split()[0]
            lname = i.split()[1]

            for j in dic:

                if i == dic[j]: 

                    time = f'["{j}"]'

                    arr = [fname, lname, time]
                    sheet.append(arr)

    wb.save("companies_Tables.xlsx")        
    return li 


In [54]:
#build dict for each company students 
# keys --> students full name 
# values --> intervel of the times they free 
def create_company_student_dict(stu_dict,company_stu_names):
    str_temp  = ""
    company_stu_dic = {}
    for i in  company_stu_names:
        if type(stu_dict [i])== type("stu_dict [i]"):
            str_temp = stu_dict [i]
            x = (str_temp.replace("] and  [", ","))
            x = x.replace(" ", "")
            x2 = ast.literal_eval(x)
            stu_dict [i]= x2
        company_stu_dic [i] = stu_dict [i]  
    return company_stu_dic



In [55]:
# create file where all the students 
# f-l names col and thired column has time when the interview
def create_company_students_interveiws_tables_sheets(sheet, company_stu_dic, company_schedule, wb):
    for i in sheet['A1:C9']:
        if i[0].value != None:
            str_temp = i[0].value + " " + i[1].value
            if str_temp in company_stu_dic: 
                i[2].value = str(get_keys_from_value(company_schedule,str_temp ))
            wb.save('extra.xlsx')


In [56]:
# create file 
# where all the students have their interiews and time  and what company
def final3(names, company_schedule, arr, sheet):
    stu_excel_file = pd.read_excel("stufree.xlsx")
    all_students = get_stu_fullname_array(stu_excel_file)

    names = remove_empty(list(company_schedule.values()))

    sample_bookEdited = openpyxl.load_workbook("sample_bookEdited.xlsx");

    abc = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


    worksheet= sample_bookEdited['schedule']


    counter = 0

    for str_temp in names:
        print(str_temp in company_schedule.values() , "--->", str_temp)
        if str_temp in company_schedule.values(): 

            s =(get_keys_from_value(company_schedule,str_temp ))
          
            columnindex = abc[arr.index(s[0])+2]

            
            str_tem_Index = str(all_students.index(str_temp) + 2)

            worksheet[str(columnindex+str_tem_Index)] = sheet.title

        counter = counter + 1
        sample_bookEdited.save("sample_bookEdited.xlsx")
   


In [57]:

#schedule_companies_students 
# (all the students given by company schdeuled here)
def schedule_companies_students():
    arr_interval_company = ["10", "10:30", "11", "11:30", "12", "12:30", "1", "1:30", "2", "2:30", "3", "3:30", "4", "4:30"]

    shutil.copy("sample_book.xlsx", "sample_bookEdited.xlsx")

    company_schedule= {}
    company_schedule_withname = {}

    stu_excel_file = pd.read_excel("stufree.xlsx")
    all_students = get_stu_fullname_array(stu_excel_file)

    wb = load_workbook("company_list.xlsx")
    arr_companies_schedule = []

    for sheet in wb.worksheets:
        
        # 1 - Get max row in list of obj for students f-l names from company sheet       
        max_row, company_stu_names = get_max_row_col_comp_stu(sheet)
        
        # 2- return each company student full names         
        company_stu_fullnames = get_company_student_fullname(company_stu_names)

        
        # 3- takes all the student - students from companies 
        # left only students that are not in companies lists

        company_stu_names = convert_comp_stud_tuble_list_name(company_stu_names)
        left = remove_student_from_all_student_arr(all_students,company_stu_names )

        

        #4 -  build dict for all the students 
        # keys --> students full name 
        # values --> intervel of the times they free in big list
        stu_dict = create_dict_fullname_time()
        
        #5-  build dict for each company students 
        # keys --> students full name 
        # values --> intervel of the times they free 
        company_stu_dic = create_company_student_dict (stu_dict,company_stu_fullnames)
        
        company_ava_dic = {i : "" for i in arr_interval_company }
        
        #6- scheduling the student in the         
        company_schedule = TieringCompany( company_stu_dic, company_ava_dic, stu_dict)
    
        # 7 -  create file fixing the company list file 
        # files will have 
        # students first name last names and time when is the interview for each company
        # in different sheet
        create_company_students_interveiws_tables_sheets(sheet,company_stu_dic, company_schedule, wb)
        
        
        # 7 - create file 
        # where all the students have their interiews and time  and what company
        # final3(company_stu_fullnames,company_schedule, arr_interval_company, sheet)

        arr_companies_schedule.append(company_schedule)
        company_schedule_withname [sheet.title] = company_schedule
        
    
    
    return left, arr_companies_schedule, company_schedule_withname


left,company_schedule, company_schedule_withname = schedule_companies_students()


In [58]:
left,company_schedule, company_schedule_withname = schedule_companies_students()
# company_schedule
len(left)
# compare company time and student 
# return student that match time in the company
def match_Time(company_time_arr ,students ):
    match_time = {}
    for i in students:
        if bool(set(students[i]) & set(company_time_arr)):
            match_time[i] = students[i]
            
        
        
    return match_time
        
# convert list of ["['']"] --> [""]
def get_time_inlist(strings):
    

    new_strings = []

    for string in strings:
        new_string = string.replace("[", "").replace("]", "").replace("'", "")
    
        new_strings.append(new_string)
    return new_strings

# get - all student left - students dict name with time ava 
def get_all_left_dict_name_time(left, stu_dict):
    
    stu_left_dict = {} 
    for key in left:
        orginal = stu_dict.pop(key).strip("][")
        orginal = orginal.split(", ")
        arr = [i.replace("'", "") for i in orginal]


        stu_left_dict[key]= arr
    return  stu_left_dict   

def dict_compare(old, new):
    arr_interval_company = ["10", "10:30", "11", "11:30", "12", "12:30", "1", "1:30", "2", "2:30", "3", "3:30", "4", "4:30"]
    company_ava_dic = {i : "" for i in arr_interval_company }
    all_sch = {}
    for i in arr_interval_company:

        if i in old.keys() and len(old[i]) >4:
            company_ava_dic [i] = old[i]
            
        elif i in new.keys() and len(new[i]) > 4:
            company_ava_dic [i] = new[i]
    return company_ava_dic
            

def schedule_rest(left):

    wb = load_workbook("extra.xlsx")
    arr_interval_company = ["10", "10:30", "11", "11:30", "12", "12:30", "1", "1:30", "2", "2:30", "3", "3:30", "4", "4:30"]
    for sheet in wb.worksheets:
        
   
        
        # 1 - Get max row in list of obj for students f-l names from company sheet       
        max_row, company_stu_names = get_max_row_col_comp_stu(sheet)
        
        # 2- return each company student full names         
        company_stu_fullnames = get_company_student_fullname(company_stu_names)

        
        #3-  build dict for all the students 
        # keys --> students full name 
        # values --> intervel of the times they free in big list
        stu_dict = create_dict_fullname_time()

        ws_dict = pd.read_excel('extra.xlsx', sheet_name=str(sheet.title))

        time_filled = get_time_inlist(list(ws_dict['Time']))
        time_notfilled = [x for x in arr_interval_company if x not in time_filled]

        
        company_ava_dic = {i : "" for i in time_notfilled }
        

       
        stu_left_dict = get_all_left_dict_name_time(left, stu_dict)
        
        #- scheduling the student in the   

        students_that_have_match_company =  match_Time(company_ava_dic ,stu_left_dict )

        company_schedule_ = TieringCompany( students_that_have_match_company, company_ava_dic, stu_left_dict)

        all_sch = dict_compare(company_schedule_withname[sheet.title], company_schedule_)

        v = list(company_schedule_.values())

        company_stu_fullnames = appendvalues( v, company_schedule_ , sheet, wb, ws_dict, left)
        
#         # 7 - create file 
#         # where all the students have their interiews and time  and what company

        final3(company_stu_fullnames,all_sch, arr_interval_company, sheet)

#         wb.save("fff2.xlsx")

        
    
    
schedule_rest(left)

True ---> Sibyl Peacock
True ---> Estefana Terrell
True ---> Saran Albers
True ---> Vanetta Forbes
True ---> Ronald Pinkerton
True ---> Samual Blank
True ---> Taryn Choate
True ---> Stephany Willard
True ---> Ligia Lancaster
True ---> Avril Boling
True ---> Estefana Terrell
True ---> Viviana Stockton
True ---> Carolann Willett
True ---> Xenia Valenti
True ---> Ronald Pinkerton
True ---> Brandee Howerton
True ---> Clair Crum
True ---> Belva Daugherty


In [59]:
def scheduling(companyList, company_ava_dic, i, stu_dict):

    if i not in company_ava_dic.values():

        if len(companyList[i])> 0:
            indexofchonesn = companyList[i].index(random.choice(companyList[i]))
            company_ava_dic[stu_dict[i].pop(0)]= i

In [60]:

def get_time_inlist(strings):
    print(strings)
    new_strings = []
    for string in strings:
        new_string = string.replace("[", "").replace("]", "").replace("'", "")
    
        new_strings.append(new_string)
    return new_strings
get_time_inlist(["['12']", "['10']", "['3:30']", "['3']", "['1']"])

["['12']", "['10']", "['3:30']", "['3']", "['1']"]


['12', '10', '3:30', '3', '1']